In [151]:
# Import libraries
import pandas as pd
from pandas_datareader import data as pdr
import time
from multiprocessing import Pool, cpu_count
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
%matplotlib inline
from greykite.framework.templates.autogen.forecast_config import ForecastConfig
from greykite.framework.templates.autogen.forecast_config import MetadataParam
from greykite.framework.templates.forecaster import Forecaster
from greykite.framework.templates.model_templates import ModelTemplateEnum
from greykite.framework.input.univariate_time_series import UnivariateTimeSeries
import zipfile
import itertools
from prophet import Prophet

In [137]:
# Read data from zip
with zipfile.ZipFile("store-sales-time-series-forecasting.zip") as z:
    
    with z.open("train.csv") as train:
        
        train = pd.read_csv(train)

    with z.open('test.csv') as test:

        test = pd.read_csv(test)

    with z.open('sample_submission.csv') as sample:

        sample = pd.read_csv(sample)

    with z.open('stores.csv') as stores:

        stores = pd.read_csv(stores)

In [138]:
# Inspect data
print(f"Unique Item Families:    {train['family'].nunique()}") 
print(f"# Stores:                {train['store_nbr'].nunique()}")
print(f"Dataset start date:      {min(train['date'])}")
print(f"Dataset end date:        {max(train['date'])}")
print(f"Test set start date:     {min(test['date'])}")
print(f"Test set end date:       {max(test['date'])}")

interval = pd.date_range(min(train['date']), max(train['date']), freq = 'd')

print(f"Num Days:               {len(interval)}")
print(f"Train Shape:            {train['date'].nunique()}")

Unique Item Families:    33
# Stores:                54
Dataset start date:      2013-01-01
Dataset end date:        2017-08-15
Test set start date:     2017-08-16
Test set end date:       2017-08-31
Num Days:               1688
Train Shape:            1684


In [139]:
# Create a master label for training 
train['item'] = train['store_nbr'].astype(str) + '_' + train['family']
test['item'] = test['store_nbr'].astype(str) + '_' + test['family']

# Drop columns
train.drop(['store_nbr', 'family'], axis = 1, inplace = True)
#test.drop(['store_nbr', 'family'], axis = 1, inplace = True)

#idx = pd.MultiIndex.from_product((interval, stores, fams), names = ['date', 'store_nbr', 'family'])

In [140]:
train.head()

,id,date,sales,onpromotion,item
0,0,2013-01-01,0.0,0,1_AUTOMOTIVE
1,1,2013-01-01,0.0,0,1_BABY CARE
2,2,2013-01-01,0.0,0,1_BEAUTY
3,3,2013-01-01,0.0,0,1_BEVERAGES
4,4,2013-01-01,0.0,0,1_BOOKS


In [141]:
test.head()

,id,date,store_nbr,family,onpromotion,item
0,3000888,2017-08-16,1,AUTOMOTIVE,0,1_AUTOMOTIVE
1,3000889,2017-08-16,1,BABY CARE,0,1_BABY CARE
2,3000890,2017-08-16,1,BEAUTY,2,1_BEAUTY
3,3000891,2017-08-16,1,BEVERAGES,20,1_BEVERAGES
4,3000892,2017-08-16,1,BOOKS,0,1_BOOKS


In [142]:
test[test['item'] == '1_AUTOMOTIVE']

,id,date,store_nbr,family,onpromotion,item
0,3000888,2017-08-16,1,AUTOMOTIVE,0,1_AUTOMOTIVE
1782,3002670,2017-08-17,1,AUTOMOTIVE,0,1_AUTOMOTIVE
3564,3004452,2017-08-18,1,AUTOMOTIVE,0,1_AUTOMOTIVE
5346,3006234,2017-08-19,1,AUTOMOTIVE,0,1_AUTOMOTIVE
7128,3008016,2017-08-20,1,AUTOMOTIVE,0,1_AUTOMOTIVE
8910,3009798,2017-08-21,1,AUTOMOTIVE,0,1_AUTOMOTIVE
10692,3011580,2017-08-22,1,AUTOMOTIVE,0,1_AUTOMOTIVE
12474,3013362,2017-08-23,1,AUTOMOTIVE,0,1_AUTOMOTIVE
14256,3015144,2017-08-24,1,AUTOMOTIVE,0,1_AUTOMOTIVE
16038,3016926,2017-08-25,1,AUTOMOTIVE,0,1_AUTOMOTIVE


In [143]:
df = train.pivot(index = 'date', columns = 'item', values = 'sales').reset_index().rename({'date': 'ds'}, axis = 1)

In [144]:
df.head()

item,ds,10_AUTOMOTIVE,10_BABY CARE,10_BEAUTY,10_BEVERAGES,10_BOOKS,10_BREAD/BAKERY,10_CELEBRATION,10_CLEANING,10_DAIRY,...,9_MAGAZINES,9_MEATS,9_PERSONAL CARE,9_PET SUPPLIES,9_PLAYERS AND ELECTRONICS,9_POULTRY,9_PREPARED FOODS,9_PRODUCE,9_SCHOOL AND OFFICE SUPPLIES,9_SEAFOOD
0,2013-01-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.000
1,2013-01-02,3.0,0.0,6.0,396.0,0.0,151.0,0.0,1110.0,101.0,...,0.0,374.531,482.0,0.0,0.0,651.292,83.0,0.0,0.0,29.214
2,2013-01-03,2.0,0.0,3.0,399.0,0.0,129.0,0.0,970.0,113.0,...,0.0,400.863,372.0,0.0,0.0,509.496,66.0,0.0,0.0,25.000
3,2013-01-04,2.0,0.0,4.0,460.0,0.0,105.0,0.0,669.0,66.0,...,0.0,310.878,324.0,0.0,0.0,332.672,57.0,0.0,0.0,11.000
4,2013-01-05,0.0,0.0,3.0,624.0,0.0,112.0,0.0,1093.0,88.0,...,0.0,447.685,461.0,0.0,0.0,510.919,84.0,0.0,0.0,28.361


In [145]:
df.shape

(1684, 1783)

In [179]:
# Create a function to perform modeling
def prophet_model(item):

    model = Prophet()
    
    trainingdata = df[['ds', item]].rename({item: 'y'}, axis = 1)
    trainingdata['y'] = trainingdata['y'].astype(float)
    
    model.fit(trainingdata)

    fut = model.make_future_dataframe(periods = 16)

    preds = model.predict(fut)[['ds', 'yhat']] 
    preds['ds'] = pd.to_datetime(preds['ds'])
    
    out = test[test['item'] == col].drop('item', axis = 1)
    out['date'] = pd.to_datetime(out['date'])
    
    out1 = out.merge(preds, how = 'left', left_on = 'date', right_on = 'ds').rename({'yhat': 'sales'}, axis = 1).drop('ds', axis = 1)

    return out1

In [180]:
num_cpus = cpu_count()

In [182]:
items = df.drop('ds', axis = 1).columns

In [183]:
items

Index(['10_AUTOMOTIVE', '10_BABY CARE', '10_BEAUTY', '10_BEVERAGES',
       '10_BOOKS', '10_BREAD/BAKERY', '10_CELEBRATION', '10_CLEANING',
       '10_DAIRY', '10_DELI',
       ...
       '9_MAGAZINES', '9_MEATS', '9_PERSONAL CARE', '9_PET SUPPLIES',
       '9_PLAYERS AND ELECTRONICS', '9_POULTRY', '9_PREPARED FOODS',
       '9_PRODUCE', '9_SCHOOL AND OFFICE SUPPLIES', '9_SEAFOOD'],
      dtype='object', name='item', length=1782)

In [ ]:
%%time 
p = Pool(num_cpus)
result = p.map(prophet_model, items)

2024-05-23 18:01:45 prophet [INFO]: Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
2024-05-23 18:01:45 cmdstanpy [DEBUG]: input tempfile: /tmp/tmp6u2k870i/n9_u75ju.json
2024-05-23 18:01:45 prophet [INFO]: Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
2024-05-23 18:01:45 prophet [INFO]: Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
2024-05-23 18:01:45 prophet [INFO]: Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
2024-05-23 18:01:45 cmdstanpy [DEBUG]: input tempfile: /tmp/tmp6u2k870i/3w61k35f.json
2024-05-23 18:01:45 cmdstanpy [DEBUG]: input tempfile: /tmp/tmp6u2k870i/t7rk0hsv.json
2024-05-23 18:01:45 cmdstanpy [DEBUG]: input tempfile: /tmp/tmp6u2k870i/4zcn04i2.json
2024-05-23 18:01:45 cmdstanpy [DEBUG]: input tempfile: /tmp/tmp6u2k870i/fz5ioxoo.json
2024-05-23 18:01:45 cmdstanpy [DEBUG]: idx 0
2024-05-23 18:01:45 cmdstanpy [DE

18:01:45 - cmdstanpy - INFO - Chain [1] start processing


2024-05-23 18:01:45 cmdstanpy [DEBUG]: input tempfile: /tmp/tmp6u2k870i/evwpg2xq.json
2024-05-23 18:01:45 cmdstanpy [DEBUG]: idx 0
2024-05-23 18:01:45 cmdstanpy [DEBUG]: running CmdStan, num_threads: None
2024-05-23 18:01:45 cmdstanpy [INFO]: Chain [1] start processing
2024-05-23 18:01:45 cmdstanpy [DEBUG]: running CmdStan, num_threads: None
2024-05-23 18:01:45 cmdstanpy [DEBUG]: CmdStan args: ['/home/ec2-user/miniconda3/envs/one/lib/python3.10/site-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=41472', 'data', 'file=/tmp/tmp6u2k870i/t7rk0hsv.json', 'init=/tmp/tmp6u2k870i/evwpg2xq.json', 'output', 'file=/tmp/tmp6u2k870i/prophet_model5lmzsbqz/prophet_model-20240523180145.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
2024-05-23 18:01:45 cmdstanpy [DEBUG]: CmdStan args: ['/home/ec2-user/miniconda3/envs/one/lib/python3.10/site-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=88322', 'data', 'file=/tmp/tmp6u2k870i/3w61k35f.json', 'init=/tmp/tmp6u2

18:01:45 - cmdstanpy - INFO - Chain [1] start processing
18:01:45 - cmdstanpy - INFO - Chain [1] start processing


2024-05-23 18:01:45 cmdstanpy [INFO]: Chain [1] start processing
2024-05-23 18:01:45 cmdstanpy [INFO]: Chain [1] start processing
2024-05-23 18:01:45 cmdstanpy [DEBUG]: input tempfile: /tmp/tmp6u2k870i/w77kz_j_.json
2024-05-23 18:01:45 cmdstanpy [DEBUG]: idx 0
2024-05-23 18:01:45 cmdstanpy [DEBUG]: running CmdStan, num_threads: None
2024-05-23 18:01:45 cmdstanpy [DEBUG]: CmdStan args: ['/home/ec2-user/miniconda3/envs/one/lib/python3.10/site-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=79266', 'data', 'file=/tmp/tmp6u2k870i/4zcn04i2.json', 'init=/tmp/tmp6u2k870i/w77kz_j_.json', 'output', 'file=/tmp/tmp6u2k870i/prophet_model9vpgzm3x/prophet_model-20240523180145.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']


...

18:07:10 - cmdstanpy - INFO - Chain [1] done processing


2024-05-23 18:07:10 cmdstanpy [INFO]: Chain [1] done processing
2024-05-23 18:07:10 cmdstanpy [DEBUG]: input tempfile: /tmp/tmp6u2k870i/4i2d72as.json
2024-05-23 18:07:10 cmdstanpy [INFO]: Chain [1] done processing


18:07:10 - cmdstanpy - INFO - Chain [1] done processing


2024-05-23 18:07:10 cmdstanpy [INFO]: Chain [1] done processing
2024-05-23 18:07:10 cmdstanpy [DEBUG]: input tempfile: /tmp/tmp6u2k870i/fvxxkbd6.json
2024-05-23 18:07:10 cmdstanpy [DEBUG]: idx 0
2024-05-23 18:07:10 cmdstanpy [DEBUG]: running CmdStan, num_threads: None
2024-05-23 18:07:10 cmdstanpy [DEBUG]: CmdStan args: ['/home/ec2-user/miniconda3/envs/one/lib/python3.10/site-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=66906', 'data', 'file=/tmp/tmp6u2k870i/th29otr7.json', 'init=/tmp/tmp6u2k870i/fvxxkbd6.json', 'output', 'file=/tmp/tmp6u2k870i/prophet_modelg0msx_8v/prophet_model-20240523180710.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']


18:07:10 - cmdstanpy - INFO - Chain [1] start processing


In [ ]:
res = pd.concat([i for i in result])